In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from sklearn.preprocessing import OneHotEncoder
import pickle
import xgboost as xgb

pd.set_option('display.max_columns', None)


In [2]:

df = pd.read_csv('/Users/afnanalamri/Renewable_Energy/raw_data/open-meteo_weather.csv')

df.head()


,time,temperature_2m (°C),relative_humidity_2m (%),wind_speed_10m (km/h),wind_speed_100m (km/h),soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),diffuse_radiation (W/m²)
0,2024-10-18T00:00,24.4,53,17.9,32.0,24.9,33.1,34.1,33.6,0.0
1,2024-10-18T01:00,24.8,52,18.0,30.9,26.6,33.1,34.1,33.6,0.0
2,2024-10-18T02:00,24.3,56,18.8,31.5,25.9,33.0,34.1,33.6,0.0
3,2024-10-18T03:00,24.0,58,17.7,30.6,25.2,32.9,34.1,33.6,0.0
4,2024-10-18T04:00,23.7,61,17.8,31.4,24.6,32.8,34.1,33.6,2.0


In [3]:
df.columns

Index(['time', 'temperature_2m (°C)', 'relative_humidity_2m (%)',
       'wind_speed_10m (km/h)', 'wind_speed_100m (km/h)',
       'soil_temperature_0_to_7cm (°C)', 'soil_temperature_7_to_28cm (°C)',
       'soil_temperature_28_to_100cm (°C)',
       'soil_temperature_100_to_255cm (°C)', 'diffuse_radiation (W/m²)'],
      dtype='object')

In [5]:
# Create a DataFrame with a specific date (2024-10-20)
df['Date'] = pd.to_datetime(df['time'])

df.set_index('Date', inplace=True)

df.index = pd.to_datetime(df.index)
df['Date'] = df.index.date

KSA_holidays = holidays.SA(years=[2024])
df['IsHoliday'] = df['Date'].apply(lambda x: 1 if x in KSA_holidays else 0)

df= df.drop(columns='time')
df.head()

,temperature_2m (°C),relative_humidity_2m (%),wind_speed_10m (km/h),wind_speed_100m (km/h),soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),diffuse_radiation (W/m²),Date,IsHoliday
Date,,,,,,,,,,,
2024-10-18 00:00:00,24.4,53,17.9,32.0,24.9,33.1,34.1,33.6,0.0,2024-10-18,0
2024-10-18 01:00:00,24.8,52,18.0,30.9,26.6,33.1,34.1,33.6,0.0,2024-10-18,0
2024-10-18 02:00:00,24.3,56,18.8,31.5,25.9,33.0,34.1,33.6,0.0,2024-10-18,0
2024-10-18 03:00:00,24.0,58,17.7,30.6,25.2,32.9,34.1,33.6,0.0,2024-10-18,0
2024-10-18 04:00:00,23.7,61,17.8,31.4,24.6,32.8,34.1,33.6,2.0,2024-10-18,0


In [6]:
df.rename(columns={
    'temperature_2m (°C)': 'Temperature',
    'relative_humidity_2m (%)': 'Humidity',
    'wind_speed_10m (km/h)': 'WindSpeed',
}, inplace=True)
df = df[['Temperature', 'Humidity', 'WindSpeed','IsHoliday' ]]
df['WindSpeed']= df['WindSpeed']/ 3.6
df

,Temperature,Humidity,WindSpeed,IsHoliday
Date,,,,
2024-10-18 00:00:00,24.4,53,4.972222,0
2024-10-18 01:00:00,24.8,52,5.000000,0
2024-10-18 02:00:00,24.3,56,5.222222,0
2024-10-18 03:00:00,24.0,58,4.916667,0
2024-10-18 04:00:00,23.7,61,4.944444,0
...,...,...,...,...
2024-10-20 19:00:00,28.5,30,8.138889,0
2024-10-20 20:00:00,27.3,35,8.388889,0
2024-10-20 21:00:00,26.5,38,8.666667,0


In [7]:
# Initialize the SA holidays for 2024
KSA_holidays = holidays.SA(years=[2024])

# Print all holidays in SA for 2024
for date, name in sorted(KSA_holidays.items()):
    print(date, name)


2024-02-22 يوم التأسيسي
2024-04-10 (تقدير) عطلة عيد الفطر
2024-04-11 (تقدير) عطلة عيد الفطر
2024-04-12 (تقدير) عطلة عيد الفطر
2024-04-13 (تقدير) عطلة عيد الفطر
2024-04-14 (تقدير ملاحظة) عطلة عيد الفطر
2024-04-15 (تقدير ملاحظة) عطلة عيد الفطر
2024-06-15 (تقدير) يوم عرفة
2024-06-16 (تقدير) عطلة عيد الأضحى
2024-06-17 (تقدير) عطلة عيد الأضحى
2024-06-18 (تقدير) عطلة عيد الأضحى
2024-06-19 (تقدير ملاحظة) عطلة عيد الأضحى
2024-09-23 اليوم الوطني


In [8]:
# Create new columns using the Datetime index
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Hour'] = df.index.hour
# df = df[['IsHoliday']]


In [9]:
# Define the seasons
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df['Season'] = df['Month'].apply(categorize_season)

In [11]:
df.head()

,Temperature,Humidity,WindSpeed,IsHoliday,Year,Month,Day,Hour,Season
Date,,,,,,,,,
2024-10-18 00:00:00,24.4,53,4.972222,0,2024,10,18,0,Autumn
2024-10-18 01:00:00,24.8,52,5.000000,0,2024,10,18,1,Autumn
2024-10-18 02:00:00,24.3,56,5.222222,0,2024,10,18,2,Autumn
2024-10-18 03:00:00,24.0,58,4.916667,0,2024,10,18,3,Autumn
2024-10-18 04:00:00,23.7,61,4.944444,0,2024,10,18,4,Autumn


In [12]:
# Define time of day categories
def categorize_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

df['TimeOfDay'] = df['Hour'].apply(categorize_time_of_day)

# df['Weekday'] = df.index.weekday
# df['IsWeekend'] = df['Weekday'].apply(lambda x: 1 if x >= 5 else 0)
df.head()

,Temperature,Humidity,WindSpeed,IsHoliday,Year,Month,Day,Hour,Season,TimeOfDay
Date,,,,,,,,,,
2024-10-18 00:00:00,24.4,53,4.972222,0,2024,10,18,0,Autumn,Night
2024-10-18 01:00:00,24.8,52,5.000000,0,2024,10,18,1,Autumn,Night
2024-10-18 02:00:00,24.3,56,5.222222,0,2024,10,18,2,Autumn,Night
2024-10-18 03:00:00,24.0,58,4.916667,0,2024,10,18,3,Autumn,Night
2024-10-18 04:00:00,23.7,61,4.944444,0,2024,10,18,4,Autumn,Night


In [13]:
df['Weekday'] = df.index.weekday  # Access weekday from index
# Add a column to indicate if it's a weekend
df['IsWeekend'] = df['Weekday'].apply(lambda x: 1 if x >= 4 else 0)

df = df.drop(columns = ['Weekday'])

In [15]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(df[['Season']])
df[ohe.get_feature_names_out(['Season'])] = ohe.transform(df[['Season']])

df['Season_Autumn']=0
df['Season_Spring']=0
df['Season_Summer']=0
df['Season_Winter']=0
ohe.fit(df[['TimeOfDay']])
df[ohe.get_feature_names_out(['TimeOfDay'])] = ohe.transform(df[['TimeOfDay']])

df = df.drop(columns = ['Season', 'TimeOfDay'])

df.head()

,Temperature,Humidity,WindSpeed,IsHoliday,Year,Month,Day,Hour,IsWeekend,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,TimeOfDay_Afternoon,TimeOfDay_Evening,TimeOfDay_Morning,TimeOfDay_Night
Date,,,,,,,,,,,,,,,,,
2024-10-18 00:00:00,24.4,53,4.972222,0,2024,10,18,0,1,0,0,0,0,0.0,0.0,0.0,1.0
2024-10-18 01:00:00,24.8,52,5.000000,0,2024,10,18,1,1,0,0,0,0,0.0,0.0,0.0,1.0
2024-10-18 02:00:00,24.3,56,5.222222,0,2024,10,18,2,1,0,0,0,0,0.0,0.0,0.0,1.0
2024-10-18 03:00:00,24.0,58,4.916667,0,2024,10,18,3,1,0,0,0,0,0.0,0.0,0.0,1.0
2024-10-18 04:00:00,23.7,61,4.944444,0,2024,10,18,4,1,0,0,0,0,0.0,0.0,0.0,1.0


In [16]:
# Lag values (2, 4, 6, ..., 48 hours)
lags = [x for x in range(2,49,2)]

# Create lag features on the grouped (mean) data
# df_grouped = df.copy()

columns_to_lag = [
    'Temperature', 'Humidity', 'WindSpeed'
]

for col in columns_to_lag:
    for lag in lags:
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

# Remove rows with NaN values due to lagging
df_lagged = df.dropna()
df_lagged.head()

,Temperature,Humidity,WindSpeed,IsHoliday,Year,Month,Day,Hour,IsWeekend,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,TimeOfDay_Afternoon,TimeOfDay_Evening,TimeOfDay_Morning,TimeOfDay_Night,Temperature_lag2,Temperature_lag4,Temperature_lag6,Temperature_lag8,Temperature_lag10,Temperature_lag12,Temperature_lag14,Temperature_lag16,Temperature_lag18,Temperature_lag20,Temperature_lag22,Temperature_lag24,Temperature_lag26,Temperature_lag28,Temperature_lag30,Temperature_lag32,Temperature_lag34,Temperature_lag36,Temperature_lag38,Temperature_lag40,Temperature_lag42,Temperature_lag44,Temperature_lag46,Temperature_lag48,Humidity_lag2,Humidity_lag4,Humidity_lag6,Humidity_lag8,Humidity_lag10,Humidity_lag12,Humidity_lag14,Humidity_lag16,Humidity_lag18,Humidity_lag20,Humidity_lag22,Humidity_lag24,Humidity_lag26,Humidity_lag28,Humidity_lag30,Humidity_lag32,Humidity_lag34,Humidity_lag36,Humidity_lag38,Humidity_lag40,Humidity_lag42,Humidity_lag44,Humidity_lag46,Humidity_lag48,WindSpeed_lag2,WindSpeed_lag4,WindSpeed_lag6,WindSpeed_lag8,WindSpeed_lag10,WindSpeed_lag12,WindSpeed_lag14,WindSpeed_lag16,WindSpeed_lag18,WindSpeed_lag20,WindSpeed_lag22,WindSpeed_lag24,WindSpeed_lag26,WindSpeed_lag28,WindSpeed_lag30,WindSpeed_lag32,WindSpeed_lag34,WindSpeed_lag36,WindSpeed_lag38,WindSpeed_lag40,WindSpeed_lag42,WindSpeed_lag44,WindSpeed_lag46,WindSpeed_lag48
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-10-20 00:00:00,24.2,48,8.361111,0,2024,10,20,0,1,0,0,0,0,0.0,0.0,0.0,1.0,25.0,26.3,28.3,28.9,32.0,33.0,31.6,28.9,25.6,23.3,24.2,24.3,25.3,25.5,26.4,27.8,31.0,32.2,31.4,29.2,26.2,23.7,24.3,24.4,45.0,39.0,32.0,35.0,25.0,25.0,32.0,40.0,50.0,59.0,61.0,63.0,58.0,47.0,42.0,34.0,25.0,29.0,32.0,40.0,52.0,61.0,56.0,53.0,9.138889,9.166667,6.777778,6.527778,6.083333,5.500000,5.444444,6.472222,8.166667,7.861111,7.166667,7.000000,6.805556,3.194444,2.388889,2.361111,4.861111,3.916667,3.083333,5.083333,6.694444,4.944444,5.222222,4.972222
2024-10-20 01:00:00,24.2,49,7.944444,0,2024,10,20,1,1,0,0,0,0,0.0,0.0,0.0,1.0,24.5,25.6,27.3,28.8,30.3,32.9,32.5,30.4,27.3,24.2,23.7,24.7,24.7,25.4,26.4,27.0,29.4,31.9,32.0,30.4,27.7,24.8,24.0,24.8,47.0,42.0,34.0,33.0,30.0,24.0,28.0,36.0,44.0,56.0,60.0,61.0,61.0,53.0,48.0,38.0,26.0,28.0,30.0,36.0,46.0,58.0,58.0,52.0,8.694444,9.583333,8.500000,6.305556,5.638889,5.638889,5.638889,5.333333,7.694444,7.527778,7.722222,7.166667,7.250000,4.833333,3.750000,2.472222,3.805556,5.055556,3.166667,3.777778,6.444444,5.277778,4.916667,5.000000
2024-10-20 02:00:00,23.8,50,7.305556,0,2024,10,20,2,1,0,0,0,0,0.0,0.0,0.0,1.0,24.2,25.0,26.3,28.3,28.9,32.0,33.0,31.6,28.9,25.6,23.3,24.2,24.3,25.3,25.5,26.4,27.8,31.0,32.2,31.4,29.2,26.2,23.7,24.3,48.0,45.0,39.0,32.0,35.0,25.0,25.0,32.0,40.0,50.0,59.0,61.0,63.0,58.0,47.0,42.0,34.0,25.0,29.0,32.0,40.0,52.0,61.0,56.0,8.361111,9.138889,9.166667,6.777778,6.527778,6.083333,5.500000,5.444444,6.472222,8.166667,7.861111,7.166667,7.000000,6.805556,3.194444,2.388889,2.361111,4.861111,3.916667,3.083333,5.083333,6.694444,4.944444,5.222222
2024-10-20 03:00:00,23.4,52,6.972222,0,2024,10,20,3,1,0,0,0,0,0.0,0.0,0.0,1.0,24.2,24.5,25.6,27.3,28.8,30.3,32.9,32.5,30.4,27.3,24.2,23.7,24.7,24.7,25.4,26.4,27.0,29.4,31.9,32.0,30.4,27.7,24.8,24.0,49.0,47.0,42.0,34.0,33.0,30.0,24.0,28.0,36.0,44.0,56.0,60.0,61.0,61.0,53.0,48.0,38.0,26.0,28.0,30.0,36.0,46.0,58.0,58.0,7.944444,8.694444,9.583333,8.500000,6.305556,5.638889,5.638889,5.638889,5.333333,7.694444,7.527778,7.722222,7.166667,7.250000,4.833333,3.750000,2.472222,3.805556,5.055556,3.166667,3.777778,6.444444,5.277778,4.916667
2024-10-20 04:00:00,23.0,53,6.777778,0,2024,10,20,4,1,0,0,0,0,0.0,0.0,0.0,1.0,23.8,24.2,25.0,26.3,28.3,28.9,32.0,33.0,31.6,28.9,25.6,23.3,24.2,24.3,25.3,25.5,26.4,27.8,31.0,32.2,31.4,29.2,26.2,23.7,50.0,48.0,45.0,39.0,32.0,35.0,25.0,25.0,32.0,40.0,50.0,59.0,61.0,63.0,58.0,47.0,42.0,34.0,25.0,29.0,32.0,40.0,52.0,61.0,7.305556,8.361111,9.138889,9.166667,6.777778,6.527778,6.083333,5.500000,5.444444,6.47222

In [17]:
df_lagged.columns

Index(['Temperature', 'Humidity', 'WindSpeed', 'IsHoliday', 'Year', 'Month',
       'Day', 'Hour', 'IsWeekend', 'Season_Autumn', 'Season_Spring',
       'Season_Summer', 'Season_Winter', 'TimeOfDay_Afternoon',
       'TimeOfDay_Evening', 'TimeOfDay_Morning', 'TimeOfDay_Night',
       'Temperature_lag2', 'Temperature_lag4', 'Temperature_lag6',
       'Temperature_lag8', 'Temperature_lag10', 'Temperature_lag12',
       'Temperature_lag14', 'Temperature_lag16', 'Temperature_lag18',
       'Temperature_lag20', 'Temperature_lag22', 'Temperature_lag24',
       'Temperature_lag26', 'Temperature_lag28', 'Temperature_lag30',
       'Temperature_lag32', 'Temperature_lag34', 'Temperature_lag36',
       'Temperature_lag38', 'Temperature_lag40', 'Temperature_lag42',
       'Temperature_lag44', 'Temperature_lag46', 'Temperature_lag48',
       'Humidity_lag2', 'Humidity_lag4', 'Humidity_lag6', 'Humidity_lag8',
       'Humidity_lag10', 'Humidity_lag12', 'Humidity_lag14', 'Humidity_lag16',
       'Humi

In [35]:
columns = ['Temperature', 'Humidity', 'WindSpeed', 'Year', 'Month', 'Day', 'Hour',
       'IsHoliday', 'IsWeekend', 'Season_Autumn', 'Season_Spring',
       'Season_Summer', 'Season_Winter', 'TimeOfDay_Afternoon',
       'TimeOfDay_Evening', 'TimeOfDay_Morning', 'TimeOfDay_Night',
       'Temperature_lag2', 'Temperature_lag4', 'Temperature_lag6',
       'Temperature_lag8', 'Temperature_lag10', 'Temperature_lag12',
       'Temperature_lag14', 'Temperature_lag16', 'Temperature_lag18',
       'Temperature_lag20', 'Temperature_lag22', 'Temperature_lag24',
       'Temperature_lag26', 'Temperature_lag28', 'Temperature_lag30',
       'Temperature_lag32', 'Temperature_lag34', 'Temperature_lag36',
       'Temperature_lag38', 'Temperature_lag40', 'Temperature_lag42',
       'Temperature_lag44', 'Temperature_lag46', 'Temperature_lag48',
       'Humidity_lag2', 'Humidity_lag4', 'Humidity_lag6', 'Humidity_lag8',
       'Humidity_lag10', 'Humidity_lag12', 'Humidity_lag14', 'Humidity_lag16',
       'Humidity_lag18', 'Humidity_lag20', 'Humidity_lag22', 'Humidity_lag24',
       'Humidity_lag26', 'Humidity_lag28', 'Humidity_lag30', 'Humidity_lag32',
       'Humidity_lag34', 'Humidity_lag36', 'Humidity_lag38', 'Humidity_lag40',
       'Humidity_lag42', 'Humidity_lag44', 'Humidity_lag46', 'Humidity_lag48',
       'WindSpeed_lag2', 'WindSpeed_lag4', 'WindSpeed_lag6', 'WindSpeed_lag8',
       'WindSpeed_lag10', 'WindSpeed_lag12', 'WindSpeed_lag14',
       'WindSpeed_lag16', 'WindSpeed_lag18', 'WindSpeed_lag20',
       'WindSpeed_lag22', 'WindSpeed_lag24', 'WindSpeed_lag26',
       'WindSpeed_lag28', 'WindSpeed_lag30', 'WindSpeed_lag32',
       'WindSpeed_lag34', 'WindSpeed_lag36', 'WindSpeed_lag38',
       'WindSpeed_lag40', 'WindSpeed_lag42', 'WindSpeed_lag44',
       'WindSpeed_lag46', 'WindSpeed_lag48']

df_lagged = df_lagged[columns]

In [45]:
my_model = pickle.load(open("model3.pkl","rb"))

In [47]:
# Filter the DataFrame for 3 PM
df_filtered = df[(df['Hour'] == 15) & (df['TimeOfDay_Afternoon'] == 1)]

# Ensure there is at least one row
if not df_filtered.empty:
    row_to_predict = df_filtered.iloc[0]  # Get the first matching row

    # Send this row to the model for prediction
    prediction = my_model.predict([row_to_predict]) * 6.359

    # Display the prediction result
    print(f"Prediction for the row at 3 PM: {prediction}")
else:
    print("No row found for 3 PM.")


Prediction for the row at 3 PM: [1.29298737]


/Users/afnanalamri/.pyenv/versions/3.10.6/envs/Renewable_Energy/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
